In [ ]:
import healpy as hp
import numpy as np
from scipy import optimize
import pandas as pd
import time

Read 100 modulated maps in NESST ordering, in this case flux threshold 1 $\mu$Jy and sample *z>0.5*. Can be changed.

In [ ]:
modmaps=[]
for i in range(0,100):
    modmaps.append(hp.read_map('path/simul1/modmapswo0_5/modmap%s.fits'%i,nest=True))


Read mask map consisting of zeros in the masked part and ones in the observable part. Also degrade the maps to $N_{side}=32$. Finally apply the mask

In [ ]:
mask=hp.read_map('/home/ixaka/TFG/mask64_planck.fits',nest=True)

uda=hp.ud_grade(mask,32,order_in='NEST',order_out='NEST')
for val in range(len(uda)):
        if uda[val]<1:
            uda[val]=0

mask_mod=[]
mask_mod_deg=[]
for m in modmaps:
    mask_mod.append(m*mask)
    mud=hp.ud_grade(m,32,order_in='NEST',order_out='NEST')
    mask_mod_deg.append(mud*uda)

Define estimator for degraded maps and pixel coordinates. This function is used to estimate both the amplitude and the direction. It returns the value of the estimator with the given input parameters. Can be used with all methods but with the mask it only works well for Nelder-Mead.

In [ ]:
def estimPix(x,mapa,average):
    pixels=[]
    a,dirp=x
    vec=hp.pix2vec(32,int(dirp),nest=True)
    for p in range(len(mapa)):
        if mapa[p] !=0:
            cosp=np.dot(vec,hp.pix2vec(32,p,nest=True))
            r=(mapa[p]-average*(1+a*cosp))**2/(average*(1+a*cosp))
            pixels.append(r)
    return np.sum(pixels)

Estimator to determine direction in Cartesian coordinates given a fixed amplitude.

In [ ]:
def estim_ampvec(x,mapa,amp,average):
    pixels=[]
    for p in range(len(mapa)):
        if mapa[p] !=0:
            cosp=np.dot(x,hp.pix2vec(64,p,nest=True))
            e=(mapa[p]-average*(1+amp*cosp))**2/(average*(1+amp*cosp))
            pixels.append(e)
    return np.sum(pixels)

Gradient of the estimator above. Returns an array with the derivatives over *x,y,z*. 

In [ ]:
def grad_ampvec(x,mapa,amp,average):
    xv=[]
    yv=[]
    zv=[]
    for p in range(len(mapa)):
        if mapa[p] != 0:
            vec=hp.pix2vec(64,p,nest=True)
            cosp=np.dot(vec,x)
            h=amp*vec[0]*average-amp*vec[0]*mapa[p]**2/(average*(amp*cosp+1)**2)
            xv.append(h)
            y=amp*vec[1]*average-amp*vec[1]*mapa[p]**2/(average*(amp*cosp+1)**2)
            yv.append(y)
            z=amp*vec[2]*average-amp*vec[2]*mapa[p]**2/(average*(amp*cosp+1)**2)
            zv.append(z)
    return np.array([np.sum(xv),np.sum(yv),np.sum(zv)])

Estimator to determine the amplitude given a fixed direction vector in Cartesian coordinates.

In [ ]:
def estim_dir(x,mapa,dirvec,average):
    pixels=[]
    for p in range(len(mapa)):
        if mapa[p] !=0:
            cosp=np.dot(dirvec,hp.pix2vec(64,p,nest=True))
            r=(mapa[p]-average*(1+x*cosp))**2/(average*(1+x*cosp))
            pixels.append(r)
    return np.sum(pixels)

Gradient of the estimator above. Returns an array with the derivative over the amplitude

In [ ]:
def grad_dir(x,mapa,dirvec,average):
    av=[]
    for p in range(len(mapa)):
        if mapa[p] != 0:
            vec=hp.pix2vec(64,p,nest=True)
            cosp=np.dot(vec,dirvec)
            a=average*cosp-(mapa[p]**2*cosp)/(average*(x*cosp+1)**2)
            av.append(a)
    return np.array([np.sum(av)])

Define initial guesses for the methods to use, they are the amplitude and direction of the CMB dipole. *ing_deg* has the amplitude and the pixel number, *ing_vec* the direction in Cartesian coordinates and *ing_amp* just the amplitude

In [ ]:
ing_deg=[4.62e-3,hp.ang2pix(32,264.021,48.253,nest=True,lonlat=True)]
ing_vec=hp.ang2vec(264.021,48.253,lonlat=True)
ing_amp=4.62e-3

Optimization with Nelder-Mead. Using *estimPix*. As arguments it has a degradated masked map and the mean value of the pixel for that map. In addition the number of the map and the time needed in that iteration are printed.

In [ ]:
nm=[]
totime=0
for m in range(100):
    tim=time.time()
    print(m)
    nm.append(optimize.minimize(estimPix,ing_deg,args=(mask_mod_deg[m],np.mean(modmaps[m])),method='Nelder-Mead'))
    tim2=time.time()-tim
    print(tim2)
    totime+=tim2
print(totime)

Create arrays with amplitude and direction and write them into files.

In [ ]:
amp_nm=[]
dire_nm=[]
for i in nm:
    amp_nm.append(i.x[0])
    dire_nm.append(i.x[1:])

newfile=pd.DataFrame(amp_nm)
newfile.to_csv('path/simul1/maskwo0_5/amp_deg_mask.dat',sep=' ',index=None,header=None)
newfile=pd.DataFrame(np.int16(dire_nm))
newfile.to_csv('path/simul1/maskwo0_5/dir_deg_mask.dat',sep=' ',index=None,header=None)


Estimation of the Cartesian coordinates of direction with L-BFGS-B. In this case, appart of the maps and the average of map, the mean value of the amplitude obtained with Nelder-Mead is given.

In [ ]:
lbf_ampFix=[]
totime1=0
for m in range(100):
    tim=time.time()
    print(m)
    lbf_ampFix.append(optimize.minimize(estim_ampvec,ing_vec,jac=grad_ampvec,args=(mask_mod[m],np.mean(amp_nm),np.mean(modmaps[m])),method='L-BFGS-B'))
    tim2=time.time()-tim
    print(tim2)
    totime1+=tim2
print(totime1)

In [ ]:
dire_lbf,l_lbf,b_lbf=[],[],[]
for i in lbf_ampFix:
    dire_lbf.append(i.x)
    la,ba=hp.vec2ang(i.x,lonlat=True)
    l_lbf.append(la)
    b_lbf.append(ba)
    
newfile=pd.DataFrame(dire_lbf)
newfile.to_csv('path/simul1/maskwo0_5/dir_mask.dat',sep=' ',index=None,header=None)
newfile=pd.DataFrame(np.hstack([l_lbf,b_lbf]))
newfile.to_csv('path/simul/maskwo0_5/dir_masklonlat.dat',sep=' ',index=None,header=None)
    

Estimation of the amplitude with Newton's conjugate gradient. Here the fixed direction vector has been obtained with the mean of the longitude and latitude from L-NFGS-B.

In [ ]:
ncg_dirFix=[]
counts=0
totime3=0
fixdir=hp.ang2vec(np.mean(l_lbf),np.mean(b_lbf),lonlat=True)
for m in range(100):
    tim=time.time()
    print(m)
    ncg_dirFix.append(optimize.minimize(estim_dir,4.62e-3,jac=grad_dir,args=(mask_mod[m],fixdir,np.mean(modmaps[m])),method='Newton-CG'))
    tim2=time.time()-tim
    print(tim2)
    totime3+=tim2
print(totime3)


In [ ]:
amp_ncg=[]
for i in ncg_dirFix:
    amp_ncg.append(i.x)
    
newfile=pd.DataFrame(amp_ncg)
newfile.to_csv('path/simul1/maskwo0_5/amp_mask.dat',sep=' ',index=None,header=None)
